In [3]:
#!pip install dash-bootstrap-components

In [1]:
 # -*- coding: utf-8 -*-

# We start with the import of standard ML librairies

import pandas as pd
import numpy as np
import math

from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

# We add all Plotly and Dash necessary librairies
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
y = np.ravel(y)

X['sq_mt_built'] = X['sq_mt_built']-16
X['buy_price/barrio'] = X['buy_price/barrio']-308

# We start by creating a virtual regression use-case
#X, y = make_regression(n_samples=1000, n_features=8, n_informative=5, random_state=22)

# We rename columns as industrial parameters
#col_names = ["Temperature","Viscosity","Pressure", "pH","Inlet_flow", "Rotating_Speed","Particles_size","Color_density"]

# We change the most important features ranges to make them look like actual figures
# df["pH"]=6.5+df["pH"]/4
# df["Pressure"]=10+df["Pressure"]
# df["Temperature"]=20+df["Temperature"]
# df["Y"] = 90+y/20

# We train a simple RF model

model = RandomForestRegressor(n_estimators = 150)
model.fit(X, y)

# We create a DataFrame to store the features' importance and their corresponding label
df_feature_importances = pd.DataFrame(model.feature_importances_*100,columns=["Importance"],index=X.columns)
df_feature_importances = df_feature_importances.sort_values("Importance", ascending=False)


# We record the name, min, mean and max of the three most important features
dropdown_1_label = df_feature_importances.index[0]
dropdown_1_min = math.floor(X[dropdown_1_label].min())
dropdown_1_mean = round(X[dropdown_1_label].mean())
dropdown_1_max = round(X[dropdown_1_label].max())

dropdown_2_label = df_feature_importances.index[1]
dropdown_2_min = math.floor(X[dropdown_2_label].min())
dropdown_2_mean = round(X[dropdown_2_label].mean())
dropdown_2_max = round(X[dropdown_2_label].max())

dropdown_3_label = df_feature_importances.index[2]
dropdown_3_min = math.floor(X[dropdown_3_label].min())
dropdown_3_mean = round(X[dropdown_3_label].mean())
dropdown_3_max = round(X[dropdown_3_label].max())


###############################################################################

app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])

# The page structure will be:
#    Features Importance Chart
#    <H4> Feature #1 name
#    Slider to update Feature #1 value
#    <H4> Feature #2 name
#    Slider to update Feature #2 value
#    <H4> Feature #3 name
#    Slider to update Feature #3 value
#    <H2> Updated Prediction
#    Callback fuction with Sliders values as inputs and Prediction as Output

# We apply basic HTML formatting to the layout
app.layout = html.Div(style={'textAlign': 'center', 'width': '700px', 'font-family': 'Verdana'},
                      
                    children=[

                        # The same logic is applied to the following names / sliders
                        html.H3(children="SIMULATION TOOL"),
                        
                                               
                        # We display the most important feature's name
                        html.H5(children=dropdown_1_label),

                        # The Dash Slider is built according to Feature #1 ranges
                        dcc.Slider(
                            id='X1_slider',
                            min=25,
                            max=dropdown_1_max,
                            step=200,
                            value=dropdown_1_mean,
                           marks={i: '{}'.format(i) for i in range(dropdown_1_min,dropdown_1_max,50)},),

                        # The same logic is applied to the following names / sliders
                        html.H5(children=dropdown_2_label),

                        dcc.Slider(
                            id='X2_slider',
                            min=2000,
                            max=dropdown_2_max,
                            step=2000,
                            value=dropdown_2_mean,
                            marks={i: '{}'.format(i) for i in range(dropdown_2_min,dropdown_2_max,500)},),
                        
                        html.H5(children=dropdown_3_label),

                        dcc.Slider(
                            id='X3_slider',
                            min=dropdown_3_min,
                            max=5,
                            step=1,
                            value=dropdown_3_mean,
                            marks={i: '{}'.format(i) for i in range(dropdown_3_min,dropdown_3_max)},
                        ),
                        
                        # The predictin result will be displayed and updated here
                        html.H4(id="prediction_result"),

                    ])

# The callback function will provide one "Ouput" in the form of a string (=children)
@app.callback(Output(component_id="prediction_result",component_property="children"),
# The values correspnding to the three sliders are obtained by calling their id and value property
              [Input("X1_slider","value"), Input("X2_slider","value"), Input("X3_slider","value")])

# The input variable are set in the same order as the callback Inputs
def update_prediction(X1, X2, X3):

    # We create a NumPy array in the form of the original features
    # ["Pressure","Viscosity","Particles_size", "Temperature","Inlet_flow", "Rotating_Speed","pH","Color_density"]
    # Except for the X1, X2 and X3, all other non-influencing parameters are set to their mean
    input_X = np.array([X3,
                        X1,
                        X2,
                        X["District_no"].mean(),
                        X["is_renewal_needed"].mean(),
                        X["has_parking"].mean(),
                        X["type"].mean(),
                       ]).reshape(1,-1)        
    
    # Prediction is calculated based on the input_X array
    prediction = round(model.predict(input_X)[0],0)
    
    # And retuned to the Output of the callback function
    return "Prediction: {}€".format(round(prediction,0))

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2020 23:00:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2020 23:00:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2020 23:00:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2020 23:00:09] "POST /_dash-update-component HTTP/1.1" 200 -
